<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/LogADEmpirical1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://zenodo.org/record/8115559

python main_run.py --data_dir ./dataset/HDFS --output_dir ./output/ --model_name DeepLog --dataset_name HDFS --grouping session --window_size 20 --step_size 1 --train_size 0.01 --is_chronological --session_level entry --log_file HDFS.log --batch_size 2048 --lr 0.001 --accumulation_step 1 --optimizer adamw --sequential --history_size 10 --embeddings embeddings_average.json --hidden_size 128 --num_layers 2 --embedding_dim 300 --topk 10 --dropout 0.1 --max_epoch 10




```
'HDFS': {  
  'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>',
  'regex': [ r'blk_-?\d+', r'(\d+\.){ 3 }\d+(:\d+)?' ]
}

'HDFS': {
  'st': 0.5,
  'depth': 4
}
```



In [1]:
!wget 'https://zenodo.org/record/8115559/files/preprocessed.zip?download=1'

--2023-07-21 14:24:26--  https://zenodo.org/record/8115559/files/preprocessed.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671284710 (640M) [application/octet-stream]
Saving to: ‘preprocessed.zip?download=1’

preprocessed.zip?do 100%[===================>] 640.19M   647KB/s    in 17m 32s 

2023-07-21 14:42:00 (623 KB/s) - ‘preprocessed.zip?download=1’ saved [671284710/671284710]



In [2]:
!unzip "/content/preprocessed.zip?download=1" -d "/content/"

Archive:  /content/preprocessed.zip?download=1
   creating: /content/BGL/
  inflating: /content/__MACOSX/._BGL  
  inflating: /content/BGL/BGL.log_structured.csv  
  inflating: /content/__MACOSX/BGL/._BGL.log_structured.csv  
  inflating: /content/BGL/BGL_embeddings.h5  
  inflating: /content/BGL/embeddings.json  
  inflating: /content/__MACOSX/BGL/._embeddings.json  
  inflating: /content/BGL/embeddings_tfidf.json  
  inflating: /content/BGL/BGL.log_templates.csv  
  inflating: /content/BGL/embeddings_average.json  
   creating: /content/HDFS/
  inflating: /content/__MACOSX/._HDFS  
  inflating: /content/HDFS/embeddings.json  
  inflating: /content/__MACOSX/HDFS/._embeddings.json  
  inflating: /content/HDFS/embeddings_tfidf.json  
  inflating: /content/HDFS/HDFS.log_templates.csv  
  inflating: /content/HDFS/anomaly_label.csv  
  inflating: /content/__MACOSX/HDFS/._anomaly_label.csv  
  inflating: /content/HDFS/embeddings_average.json  
  inflating: /content/HDFS/HDFS.log_structured.

In [4]:
!wget 'https://zenodo.org/record/8115559/files/raw.zip?download=1'

--2023-07-21 14:50:07--  https://zenodo.org/record/8115559/files/raw.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348931642 (333M) [application/octet-stream]
Saving to: ‘raw.zip?download=1’

raw.zip?download=1  100%[===================>] 332.77M   522KB/s    in 10m 53s 

2023-07-21 15:01:02 (522 KB/s) - ‘raw.zip?download=1’ saved [348931642/348931642]



In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip "/content/raw.zip?download=1" -d "/content/"

Archive:  /content/raw.zip?download=1
  inflating: /content/Thunderbird.log  
   creating: /content/HDFS/
  inflating: /content/HDFS/anomaly_label.csv  
  inflating: /content/HDFS/HDFS.log  
  inflating: /content/BGL.log        
  inflating: /content/Spirit.log     


# Parsing

In [1]:

import sys
sys.path.append('/content/') #to be able for importing libarary(drain)from this address
import Drain
import pandas as pd
import numpy as np


In [10]:
#copy HDFS.log from google drive to current directory

!cp '/content/drive/MyDrive/HDFS/RAW_HDFS/HDFS.log' '/content/'


In [14]:
#Take a quick look at the first lines of the raw log:

with open('/content/HDFS.log', "r") as file:
  n = 0
  for line in file:
    print(line)
    n += 1
    if n == 5:
      break

081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010

081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906

081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010

081109 203519 145 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010

081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-1608999687919862906 terminating



regex list: This list contains regular expressions used for optional preprocessing of log messages before parsing. Each regular expression in the list is applied to log messages to replace specific patterns with placeholders, simplifying the parsing process.

r'blk_-?\d+': This regular expression matches patterns like "blk_12345" or "blk_-6789" where "blk_" is followed by an optional "-" and then a sequence of digits. These patterns are often block IDs in HDFS logs, and they are replaced with a placeholder.

r'(\d+.){3}\d+(:\d+)?': This regular expression matches IP addresses with optional port numbers. For example, it matches patterns like "192.168.1.1:8080" or "10.0.0.1". IP addresses are common in log messages and are replaced with a placeholder as well.

In [20]:

input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'HDFS.log'  # The input log file name
log_format = "<Date> <Time> <Pid> <Level> <Component>: <Content>"  # HDFS log format
# Regular expression list for optional preprocessing (default: [])

regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

#keep_para=True >> we will also have ParameterList column contains parameters which are removed
parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex, keep_para=True)
parser.parse(log_file)


Parsing file: /content/HDFS.log
Total size after encoding is 11175629 11175629
Parsing done. [Time taken: 0:32:31.827373]


In [21]:
df_parsedlog = pd.read_csv('/content/Drain_result/HDFS.log_structured.csv')
df_parsedlog.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,[]
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,[]


In [2]:
df_parsedlog2 = pd.read_csv('/content/drive/MyDrive/HDFS/HDFS.log_structured.csv')
df_parsedlog2.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,bbb51b95,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.19.102:54..."
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,['/mnt/hadoop/mapred/system/job_200811092030_0...
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,bbb51b95,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.10.6:4052..."
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,bbb51b95,Receiving block <*> src: /<*> dest: /<*>,"['blk_-1608999687919862906', '10.250.14.224:42..."
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,"['1', 'blk_-1608999687919862906 terminating']"


# Generate_embeddings

In [ ]:
!wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip & unzip crawl-300d-2M.vec.zip'


In [ ]:
import sys

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from typing import List
from time import time
import json

The r'\w+' regular expression matches sequences of alphanumeric characters and underscores. This means that any word containing only letters, digits, or underscores will be treated as a token, and all other characters (e.g., punctuation) will be ignored.

In [ ]:
# EXAMPLE
import re

text = "The quick:brown_fox_jumps dog@19 #x!sd877 "
matches = re.findall(r'\w+', text)
print(matches)

In [ ]:
print("Loading word2vec model...")
st = time()
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./crawl-300d-2M.vec', binary=False)
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
print("Loaded word2vec model in {:.2f} seconds".format(time() - st))

In [ ]:
# remove stop word and  punctuation, split by camel case
def clean_template(template: str, remove_stop_words: bool = True):
    template = " ".join([word.lower() if word.isupper() else word for word in template.strip().split()])
    template = re.sub('[A-Z]', lambda x: " " + x.group(0), template)  # camel case
    word_tokens = tokenizer.tokenize(template)  # tokenize
    word_tokens = [w for w in word_tokens if not w.isdigit()]  # remove digital
    if remove_stop_words:  # remove stop words, we can close this function
        filtered_sentence = [w.lower() for w in word_tokens if w not in stop_words]
    else:
        filtered_sentence = [w.lower() for w in word_tokens]

    template_clean = " ".join(filtered_sentence)
    return template_clean  # return string
